In [1]:
import pulp
import csv
import pandas as pd
from dfply import *

## Data

Import the Datasets

In [2]:
## data from https://github.com/vaastav/Fantasy-Premier-League
# path_to_files = "C:\\Users\\Tom\\Documents\\Blog\\2021\\Fantasy Football"

gw_df = pd.read_csv("merged_gw.csv")
gw_df

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Aaron_Cresswell_376,0,0,7,0,1.5,376,8,5,0,...,0.0,0.0,0,0,0,0,50,True,0,1
1,Aaron_Lennon_430,0,0,3,0,0.0,430,3,0,0,...,3.0,0.0,1,0,0,0,50,True,0,1
2,Aaron_Mooy_516,0,0,0,0,0.0,516,7,0,0,...,0.0,0.0,0,0,0,0,50,False,0,1
3,Aaron_Ramsdale_494,0,0,11,0,0.0,494,2,1,0,...,1.0,0.0,2,0,0,0,45,True,0,1
4,Aaron_Wan-Bissaka_122,0,2,34,1,16.1,122,9,0,0,...,4.0,2.0,8,0,0,0,55,True,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22555,Youri_Tielemans_448,0,0,11,0,37.4,448,376,1,0,...,0.0,19.0,2,-3103,1905,5008,61,True,0,47
22556,Yves_Bissouma_53,0,0,22,0,1.3,53,372,1,1,...,1.0,7.0,7,77,147,70,50,False,0,47
22557,Zech_Medley_636,0,0,0,0,0.0,636,371,0,0,...,3.0,0.0,0,153,325,172,40,True,0,47
22558,Çaglar_Söyüncü_164,0,0,0,0,0.0,164,376,0,0,...,0.0,0.0,0,-95515,165,95680,49,True,0,47


- Remove all unnecessary columns
- Subset to every player who is there from GW1
- Get the final score of those players


In [3]:
## Remove all unnecessary columns
gw_trim = gw_df[["name", "total_points", "GW", "value"]]

## Dfply tutorial
## https://towardsdatascience.com/dplyr-style-data-manipulation-with-pipes-in-python-380dcb137000
## Dfply docs
## https://github.com/kieferk/dfply

## Subset to every player who is there from GW1
gw_trim = (gw_trim >>
    group_by(X.name) >>
    mutate(firstWeek = X.GW.min()) >>
    ungroup()
       )

gw_trim = gw_trim[gw_trim["firstWeek"] == 1]


## Get the final score of those players
gw_trim = (gw_trim >>
    group_by(X.name) >>
    mutate(finalScore = X.total_points.sum()) >>
    ungroup()
       )

# Set value for each player to their GW1 value
# Just subset to GW1 rows
gw_trim = gw_trim[gw_trim["GW"] == 1]
gw_final = gw_trim.drop(["total_points", "GW", "firstWeek"], 1)

gw_final

,name,value,finalScore
0,Aaron_Cresswell_376,50,79
1,Aaron_Lennon_430,50,15
2,Aaron_Mooy_516,50,80
3,Aaron_Ramsdale_494,45,126
4,Aaron_Wan-Bissaka_122,55,127
...,...,...,...
521,Yoshinori_Muto_260,55,9
522,Youri_Tielemans_448,65,117
523,Yves_Bissouma_53,50,39
524,Çaglar_Söyüncü_164,45,120


- Add second dataset and combine with price, pos, and team

In [4]:
pl_df = pd.read_csv("players_raw.csv")
pl_trim = pl_df[["first_name",  "second_name", "element_type",  "id",  "team"]]
pl_trim

,first_name,second_name,element_type,id,team
0,Shkodran,Mustafi,2,1,1
1,Héctor,Bellerín,2,2,1
2,Sead,Kolasinac,2,3,1
3,Ainsley,Maitland-Niles,2,4,1
4,Sokratis,Papastathopoulos,2,5,1
...,...,...,...,...,...
661,Oskar,Buur,2,587,20
662,Ryan,Giles,2,607,20
663,Bruno André,Cavaco Jordao,3,609,20
664,Daniel,Castelo Podence,3,619,20


In [5]:
pl_trim["id"] = pl_trim.loc[:,"id"].astype(str)
# pl_trim[["id"]] = pl_trim[["id"]].astype(str)
# pl_trim["id"] = pl_trim["id"].astype(str)
# pl_trim["id"] = pl_trim["id"].map(str)
# pl_trim["id"] = pl_trim["id"].apply(str)
# pl_trim.id = pl_trim.id.astype('str')
# pl_trim.to_string(columns = "id")


pl_trim = (pl_trim >>
           mutate(name = X.first_name + "_" + X.second_name + "_" + X.id) >>
           mutate(cleanName = X.first_name + " " + X.second_name) >>
           drop(X.first_name, X.second_name, X.id)
          )
pl_trim

<ipython-input-5-eb02152aadf8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl_trim["id"] = pl_trim.loc[:,"id"].astype(str)


,element_type,team,name,cleanName
0,2,1,Shkodran_Mustafi_1,Shkodran Mustafi
1,2,1,Héctor_Bellerín_2,Héctor Bellerín
2,2,1,Sead_Kolasinac_3,Sead Kolasinac
3,2,1,Ainsley_Maitland-Niles_4,Ainsley Maitland-Niles
4,2,1,Sokratis_Papastathopoulos_5,Sokratis Papastathopoulos
...,...,...,...,...
661,2,20,Oskar_Buur_587,Oskar Buur
662,2,20,Ryan_Giles_607,Ryan Giles
663,3,20,Bruno André_Cavaco Jordao_609,Bruno André Cavaco Jordao
664,3,20,Daniel_Castelo Podence_619,Daniel Castelo Podence


- Combine the two dfs

In [6]:
combined_df = (gw_final >> left_join(pl_trim, by = "name"))
combined_df

,name,value,finalScore,element_type,team,cleanName
0,Aaron_Cresswell_376,50,79,2,19,Aaron Cresswell
1,Aaron_Lennon_430,50,15,3,5,Aaron Lennon
2,Aaron_Mooy_516,50,80,3,4,Aaron Mooy
3,Aaron_Ramsdale_494,45,126,1,3,Aaron Ramsdale
4,Aaron_Wan-Bissaka_122,55,127,2,12,Aaron Wan-Bissaka
...,...,...,...,...,...,...
521,Yoshinori_Muto_260,55,9,4,13,Yoshinori Muto
522,Youri_Tielemans_448,65,117,3,9,Youri Tielemans
523,Yves_Bissouma_53,50,39,3,4,Yves Bissouma
524,Çaglar_Söyüncü_164,45,120,2,9,Çaglar Söyüncü


# Prep complete, now for Pulp
----
# Objective Function
Add the score of all players to get maximum.

# Constraints
- No more than 3 players from one team
- Budget is 100m (in df in 100,000s - e.g. 10 = 1m)
- 1 GK
- 3-5 Def
- 2-5 Mid
- 1-3 Str

# Optional
- Set it to allow 12 players, with a maximum of 11 unique players. This way it will choose the higheset scoring player twice (as if he was selected as captain). Alternatively, keep it at 11 and double his score manually.

In [7]:
x = pulp.LpVariable.dict("player", range(0, len(combined_df)),
                        0,1, cat=pulp.LpInteger)

In [8]:
prob = pulp.LpProblem("FantasyFootball", pulp.LpMaximize)

In [9]:
prob += pulp.lpSum(combined_df["finalScore"][i] * x[i] for i in range(0, len(combined_df)))

In [10]:
# Add constraints

# No more than 3 players per team
for team in combined_df.team.unique():
        prob += sum(x[i] for i in range(0, len(combined_df)) if combined_df.team[i] == team) <= 3  # max 3 players

        
# One goalkeeper
prob  += sum(x[i] for i in range(0, len(combined_df)) if combined_df["element_type"][i] == 1) == 1


# Three to Five Defenders
prob  += sum(x[i] for i in range(0, len(combined_df)) if combined_df["element_type"][i] == 2) >= 3
prob  += sum(x[i] for i in range(0, len(combined_df)) if combined_df["element_type"][i] == 2) <= 5

# Three to Five Midfielders
prob  += sum(x[i] for i in range(0, len(combined_df)) if combined_df["element_type"][i] == 3) >= 3
prob  += sum(x[i] for i in range(0, len(combined_df)) if combined_df["element_type"][i] == 3) <= 5

# One to Three Attackers
prob  += sum(x[i] for i in range(0, len(combined_df)) if combined_df["element_type"][i] == 4) >= 1
prob  += sum(x[i] for i in range(0, len(combined_df)) if combined_df["element_type"][i] == 4) <= 3

# 100MM budget (100mil in dataset is 1000)
# This does not account for subs however
# Decrease budget by the price of the cheapest player in each pos
# 4 for GK, 4 for DEF, 5 for MID, 4.5 for FW - 17.5 total
prob += sum(x[i] * combined_df["value"][i] for i in range(0, len(combined_df))) <= 825  # total cost


# Max 11 players
prob += sum(x[i] for i in range(0, len(combined_df))) ==  11


# Is limiting each player to appearing once needed?

In [11]:
prob.solve()

1

In [12]:
pulp.LpStatus[prob.status]

'Optimal'

In [13]:
print(pulp.value(prob.objective)) # Scores 2170 points

2170.0


In [14]:
for i in range(0, len(combined_df)):
    if pulp.value(x[i]) != 0:
        print("{player} makes it into the team! He scored {points} points.".format(player = combined_df["cleanName"][i], points = combined_df["finalScore"][i]))
        

Andrew Robertson makes it into the team! He scored 181 points.
Anthony Martial makes it into the team! He scored 200 points.
Danny Ings makes it into the team! He scored 198 points.
Jamie Vardy makes it into the team! He scored 210 points.
Kevin De Bruyne makes it into the team! He scored 251 points.
Matt Doherty makes it into the team! He scored 167 points.
Nick Pope makes it into the team! He scored 170 points.
Raúl Jiménez makes it into the team! He scored 194 points.
Sadio Mané makes it into the team! He scored 221 points.
Trent Alexander-Arnold makes it into the team! He scored 210 points.
Willian Borges Da Silva makes it into the team! He scored 168 points.


In [22]:
squad = pd.DataFrame()
for i in range(0, len(combined_df)):
    if pulp.value(x[i]) != 0:
        squad = squad.append(combined_df[i:i+1], ignore_index = True)
        
squad = squad.sort_values(by = ["element_type"])

In [23]:
# Double the best player's score (captain)
squad["finalScore"][squad["finalScore"] == max(squad["finalScore"])] = 2 * max(squad["finalScore"])
squad["cleanName"][squad["finalScore"] == max(squad["finalScore"])] = squad["cleanName"][squad["finalScore"] == max(squad["finalScore"])] + "(C)"

<ipython-input-23-89d3afa5e078>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  squad["finalScore"][squad["finalScore"] == max(squad["finalScore"])] = 2 * max(squad["finalScore"])
<ipython-input-23-89d3afa5e078>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  squad["cleanName"][squad["finalScore"] == max(squad["finalScore"])] = squad["cleanName"][squad["finalScore"] == max(squad["finalScore"])] + "(C)"


In [24]:
squad

,name,value,finalScore,element_type,team,cleanName
6,Nick_Pope_93,45,170,1,5,Nick Pope
0,Andrew_Robertson_181,70,181,2,10,Andrew Robertson
5,Matt_Doherty_401,60,167,2,20,Matt Doherty
9,Trent_Alexander-Arnold_182,70,210,2,10,Trent Alexander-Arnold
1,Anthony_Martial_239,75,200,3,12,Anthony Martial
4,Kevin_De Bruyne_215,95,502,3,11,Kevin De Bruyne(C)
8,Sadio_Mané_192,115,221,3,10,Sadio Mané
10,Willian_Borges Da Silva_113,70,168,3,6,Willian Borges Da Silva
2,Danny_Ings_313,60,198,4,16,Danny Ings
3,Jamie_Vardy_166,90,210,4,9,Jamie Vardy


In [27]:
def printSquad(playerList):
    
    squad = playerList
    
    print("{0:{fill}{align}{n}}".format("", fill = "-", align = ">", n=100))
    
    # Goalie
    for i in range(0, len(squad)):
        if squad["element_type"][i] == 1:
            print("{:^100s}".format(squad["cleanName"][i]))
            print("{:^100s}".format(str(squad["finalScore"][i])))
            
    print("\n")

    # Defenders
    defs = squad[squad.element_type == 2].copy().reset_index(drop = True)
    print("{:^0s} {:^66s} {:^40s}".format(str(defs["cleanName"][0]), 
                                                          str(defs["cleanName"][1]), 
                                                          str(defs["cleanName"][2])))
    print("{:^15s} {:^66s} {:^40s}".format(str(defs["finalScore"][0]), 
                                                          str(defs["finalScore"][1]), 
                                                          str(defs["finalScore"][2])))
    print("\n")
    
    
    # Mids
    mids = squad[squad.element_type == 3].copy().reset_index(drop = True)
    print("{:^30s} {:^30s} {:^10s} {:^35s}".format(str(mids["cleanName"][0]),
                                                   str(mids["cleanName"][1]),
                                                   str(mids["cleanName"][2]),
                                                   str(mids["cleanName"][3])))
    print("{:^30s} {:^30s} {:^10s} {:^35s}".format(str(mids["finalScore"][0]),
                                                   str(mids["finalScore"][1]), 
                                                   str(mids["finalScore"][2]),
                                                   str(mids["finalScore"][3])))
    print("\n")
    
    # FWs
    forwards = squad[squad.element_type == 4].copy().reset_index(drop = True)
    print("{:^60s} {:^30s}".format(str(forwards["cleanName"][0]),
                                                   str(forwards["cleanName"][1])))
    print("{:^60s} {:^30s}".format(str(forwards["finalScore"][0]),
                                                   str(forwards["finalScore"][1])))
    print("{:^100s}".format(str(forwards["cleanName"][2])))
    print("{:^100s}".format(str(forwards["finalScore"][2])))

In [28]:
printSquad(squad)
print("Total points: {points}".format(points = squad["finalScore"].sum()))

----------------------------------------------------------------------------------------------------
                                             Nick Pope                                              
                                                170                                                 


Andrew Robertson                            Matt Doherty                                     Trent Alexander-Arnold         
      181                                      167                                                   210                   


       Anthony Martial               Kevin De Bruyne(C)       Sadio Mané       Willian Borges Da Silva      
             200                            502                  221                     168                


                         Danny Ings                                   Jamie Vardy          
                            198                                           210              
                                        